In [144]:
import smartsheet
import pandas as pd 
import openpyxl
from cryptography.fernet import Fernet
import os

In [145]:
#Excel file path
#Excel_file_path = r"C:\Users\abinesh.seelan\Documents\smartsheet_API\smartsheet_excel\excel1.xlsx"

#Smartsheet folder 
Abinesh_smartsheet_id = '8801184153659268'

#new smartsheet to create
new_smartsheet_name = 'Smartsheet_imported'

#load excel to pandas dataframe
source_sheet = pd.read_excel(r"C:\Users\abinesh.seelan\Documents\smartsheet_API\smartsheet_excel\excel1.xlsx",engine='openpyxl')
print(source_sheet)






    c1r1   c2r1   c3r1   c4r1   c5r1   c6r1   c7r1   c8r1
0   c1r2   c2r2   c3r2   c4r2   c5r2   c6r2   c7r2   c8r2
1   c1r3   c2r3   c3r3   c4r3   c5r3   c6r3   c7r3   c8r3
2   c1r4   c2r4   c3r4   c4r4   c5r4   c6r4   c7r4   c8r4
3   c1r5   c2r5   c3r5   c4r5   c5r5   c6r5   c7r5   c8r5
4   c1r6   c2r6   c3r6   c4r6   c5r6   c6r6   c7r6   c8r6
5   c1r7   c2r7   c3r7   c4r7   c5r7   c6r7   c7r7   c8r7
6   c1r8   c2r8   c3r8   c4r8   c5r8   c6r8   c7r8   c8r8
7   c1r9   c2r9   c3r9   c4r9   c5r9   c6r9   c7r9   c8r9
8  c1r10  c2r10  c3r10  c4r10  c5r10  c6r10  c7r10  c8r10


In [146]:
token = bytes(os.environ.get("SMARTSHEET_TOKEN"),'utf-8')

In [147]:
key = os.environ.get("SMARTSHEET_KEY")

In [148]:
fernet_obj = Fernet(key)
smartsheet_token = (fernet_obj.decrypt(token).decode("utf-8"))
smart = smartsheet.Smartsheet(access_token=smartsheet_token)
smart.errors_as_exceptions(True)


In [149]:
def smartsheet_column(list):
    column_dict_list = []
    for column_name, i in zip(list, range(len(list))):
        column_dict = {}
        if i == 0:
            column_dict['primary'] = True
        column_dict['title'] = column_name
        column_dict['type'] = 'TEXT_NUMBER'
        column_dict_list.append(column_dict)
    return column_dict_list

In [150]:
#grab all the names of the columns from the excel sheet
columns = smartsheet_column(source_sheet.columns)

In [151]:
#Create the sheet specifications
sheet_spec = smartsheet.models.Sheet({
  'name': 'new sheet1',
  'columns': columns
})

In [152]:
#create the new folder and store the information about the new sheet in a variable called 'new_sheet'
response = smart.Folders.create_sheet_in_folder(
  Abinesh_smartsheet_id,       # folder_id
  sheet_spec)
new_sheet = response.result

In [153]:
#take the sheet_id just created and assign it to a variable called sheet_id
sheet_id = new_sheet.to_dict().get('id')

In [154]:
#create a list of column_ids
column_id_list = []
for i in new_sheet.to_dict().get('columns'):
    column_id_list.append(i.get('id'))

In [155]:
#convert the pandas dataframe information to a list of lists, with each list have information from each row.
rows = source_sheet.values.tolist()

In [156]:
#take the row information, column ids and sheet id and upload them to smartsheet.
for row in rows:
    new_row = smart.models.Row()
    new_row.to_bottom = True
    for item, column_id in zip(row, column_id_list):
        new_row.cells.append({
            'column_id': column_id,
            'value':item,
            'strict': False
        })
    response = smart.Sheets.add_rows(sheet_id, [new_row])

In [157]:
#get cloumns
response = smart.Sheets.get_columns(
  2764093019973508,       # sheet_id
  include_all=True)
columns = response.data
columns


In [158]:
#get sheet information


In [251]:
row_update = smart.models.Row()
row_update.id = 2253000191174532
row_update.parentId =2243494354806660
toTop = True
    
updated_row = smart.Sheets.update_rows(2764093019973508,[row_update])

In [220]:
sheet = smart.Sheets.get_sheet(2764093019973508)           # sheet_id
print(sheet.rows[4])

{"cells": [{"columnId": 5888876854175620, "displayValue": "2", "value": 2.0}, {"columnId": 3649714042234756, "displayValue": "c1r6", "value": "c1r6"}, {"columnId": 8153313669605252, "displayValue": "c2r6", "value": "c2r6"}, {"columnId": 834964275128196, "displayValue": "c3r6", "value": "c3r6"}, {"columnId": 5338563902498692, "displayValue": "c4r6", "value": "c4r6"}, {"columnId": 3086764088813444, "displayValue": "c5r6", "value": "c5r6"}, {"columnId": 7590363716183940, "displayValue": "c6r6", "value": "c6r6"}, {"columnId": 1960864181970820, "displayValue": "c7r6", "value": "c7r6"}, {"columnId": 6464463809341316, "displayValue": "c8r6", "value": "c8r6"}], "createdAt": "2022-03-28T15:45:02+00:00Z", "expanded": true, "id": 2253000191174532, "modifiedAt": "2022-03-29T16:56:05+00:00Z", "rowNumber": 5, "siblingId": 2243494354806660}


In [250]:

sheet = smart.Sheets.get_sheet(2764093019973508)           # sheet_id
#print(sheet.to_dict().get("columns"))
column_matrix=pd.DataFrame.from_dict(sheet.to_dict().get("columns"))
column_matrix

,autoNumberFormat,id,index,systemColumnType,title,type,validation,version,width,primary
0,"{'fill': '0', 'prefix': '', 'suffix': ''}",5888876854175620,0,AUTO_NUMBER,Hierarchy,TEXT_NUMBER,False,0,150,NaN
1,NaN,3649714042234756,1,NaN,c1r1,TEXT_NUMBER,False,0,150,True
2,NaN,8153313669605252,2,NaN,c2r1,TEXT_NUMBER,False,0,150,NaN
3,NaN,834964275128196,3,NaN,c3r1,TEXT_NUMBER,False,0,150,NaN
4,NaN,5338563902498692,4,NaN,c4r1,TEXT_NUMBER,False,0,150,NaN
5,NaN,3086764088813444,5,NaN,c5r1,TEXT_NUMBER,False,0,150,NaN
6,NaN,7590363716183940,6,NaN,c6r1,TEXT_NUMBER,False,0,150,NaN
7,NaN,1960864181970820,7,NaN,c7r1,TEXT_NUMBER,False,0,150,NaN
8,NaN,6464463809341316,8,NaN,c8r1,TEXT_NUMBER,False,0,150,NaN


In [ ]:
sheet = smart.Sheets.get_sheet(2764093019973508) 
#print(sheet)
print(sheet.to_dict().get("rows"))

In [ ]:
#method 1 to update parent-child
row_spec =  smart.models.Row({
    "id" : '2253000191174532',
    "displayValue": "c1r6a",
    "value": "c1r6"
})

response = smart.Sheets.update_rows(2764093019973508,row_spec)

updated_row = response.result

In [ ]:
#method 2 to update parent-child
row_spec =  smart.models.Row({
  "id": 2253000191174532,
  "sheetId": 2764093019973508,
  "rowNumber": 5,
  "parentId": 2243494354806660,
  "version": 0,
}
updated_row = smart.Sheets.update_rows(2764093019973508,row_spec)

In [239]:
#method 3 to update parent-child
new_cell = smart.models.Cell()
new_cell.column_id = 3649714042234756
new_cell.value = "new value1"
new_cell.strict = False

# Build the row to update
new_row = smart.models.Row()
new_row.id = 7937865267996548
new_row.cells.append(new_cell)

# Update rows
updated_row = smart.Sheets.update_rows(
  2764093019973508,      # sheet_id
  [new_row])

In [257]:
#Method 4 to update parent-child
update_cell = smart.models.Cell()
update_cell.column_id = 3649714042234756
#update_cell.value = "clr6c"

# Build the row to update
update_row = smart.models.Row()
update_row.id = 2253000191174532
updated_row.parentID =2243494354806660
#update_row.cells.append(update_cell)

# Update rows
updated_row = smart.Sheets.update_rows(
  2764093019973508,      # sheet_id
  [update_row])